# Metadata

**L1 Taxonomy** - Backend Development

**L2 Taxonomy** - Memory Management

**Subtopic** - Memory Optimization Techniques in Python

**Use Case** - Implement a Python script that uses 'gc' module to manually control garbage collection and optimize memory usage. The script should analyze the memory footprint before and after the garbage collection, providing a clear report of memory saved.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt

Develop a Python memory profiler system that uses the gc module to manually control garbage collection and optimize memory usage during function execution. The system should analyze memory footprint before and after operations, providing detailed reports of memory saved through intelligent garbage collection timing. This addresses real-world scenarios in production applications where memory leaks and inefficient garbage collection patterns can cause performance degradation, particularly in long-running services, data processing pipelines, and memory-constrained environments where precise memory management is critical for system stability.

## Input Format

- functions: List of callable functions to profile
- config: Dictionary with profiling configuration
  - auto_gc: Boolean, whether to trigger garbage collection before/after
  - detailed_stats: Boolean, include GC statistics in output  
  - track_peak: Boolean, monitor peak memory during execution
  - threshold_mb: Float, memory threshold for warnings

## Output Format

Returns ProfileReport object containing:
- function_name: String, name of profiled function
- execution_time: Float, execution time in seconds
- memory_before: Float, memory usage before execution (MB)
- memory_after: Float, memory usage after execution (MB)
- memory_saved: Float, memory freed by garbage collection (MB)
- gc_objects_collected: Integer, number of objects collected
- exceeded_threshold: Boolean, whether memory exceeded threshold

## Example 1

```python
@memory_profiler(auto_gc=True, threshold_mb=100)
def process_data():
    data = [[i]*1000 for i in range(1000)]
    return len(data)

# Output: ProfileReport(function_name='process_data', execution_time=0.045,
#         memory_before=50.2, memory_after=45.1, memory_saved=5.1,
#         gc_objects_collected=1000, exceeded_threshold=False)
```

## Example 2

```python
@memory_profiler(detailed_stats=True, track_peak=True)  
def create_circular_refs():
    objects = []
    for i in range(100):
        obj = {'id': i, 'refs': objects}
        objects.append(obj)
    return objects

# Output: ProfileReport(function_name='create_circular_refs', execution_time=0.012,
#         memory_before=25.3, memory_after=22.8, memory_saved=2.5,
#         gc_objects_collected=200, exceeded_threshold=False)
```



# Requirements

### Explicit Requirements
- Implement memory_profiler decorator using Python's gc module
- Capture memory usage before and after function execution using psutil
- Manually trigger garbage collection when auto_gc=True
- Calculate and report memory savings from garbage collection
- Support configurable memory threshold warnings
- Return structured ProfileReport with all memory metrics
- Handle nested decorator usage for combined time/memory profiling

### Implicit Requirements
- Decorator should not significantly impact function performance
- Memory measurements should be accurate and consistent
- Support both simple and detailed profiling modes
- Gracefully handle functions that don't allocate significant memory
- Provide meaningful output for debugging memory issues

## Constraints
- Functions must be callable objects
- Memory measurements are limited to process-level granularity
- GC collection may not immediately free all unused memory
- Peak memory tracking adds minimal overhead
- Threshold values must be positive numbers
- Configuration dictionary must contain valid boolean/numeric values

## Edge Cases
- Functions that allocate no additional memory
- Functions with extremely short execution times (<1ms)
- Memory usage that decreases during function execution
- Functions that raise exceptions during execution
- Recursive functions or deeply nested calls
- Functions that explicitly disable garbage collection
- Memory threshold of 0 or negative values

In [ ]:
# code

"""A memory profiler using the gc module."""
import gc
import psutil
import os
import time
from functools import wraps
from dataclasses import dataclass
from typing import Callable, Optional, Dict, Any, List


@dataclass
class ProfileReport:
    """Data class to store memory profiling results."""

    function_name: str
    execution_time: float
    memory_before: float
    memory_after: float
    memory_saved: float
    gc_objects_collected: int
    exceeded_threshold: bool
    peak_memory: Optional[float] = None
    gc_stats: Optional[Dict[str, Any]] = None

    def __repr__(self):
        """Format output to match expected format."""
        parts = [
            f"function_name='{self.function_name}'",
            f"execution_time={self.execution_time:.3f}",
            f"memory_before={self.memory_before:.1f}",
            f"memory_after={self.memory_after:.1f}",
            f"memory_saved={self.memory_saved:.1f}",
            f"gc_objects_collected={self.gc_objects_collected}",
            f"exceeded_threshold={self.exceeded_threshold}",
        ]

        if self.peak_memory is not None:
            parts.append(f"peak_memory={self.peak_memory:.1f}")

        return f"ProfileReport({', '.join(parts)})"


def memory_profiler(
    auto_gc: bool = False,
    detailed_stats: bool = False,
    track_peak: bool = False,
    threshold_mb: float = float("inf"),
) -> Callable:
    """Profile memory usage of a function."""
    if threshold_mb <= 0:
        threshold_mb = float("inf")

    def decorator(func: Callable) -> Callable:
        @wraps(func)
        def wrapper(*args, **kwargs):
            process = psutil.Process(os.getpid())

            gc_stats_before = None
            if detailed_stats:
                gc_stats_before = {
                    "generations": gc.get_count(),
                    "stats": (
                        gc.get_stats() if hasattr(gc, "get_stats") else None
                    ),
                    "garbage": len(gc.garbage),
                    "thresholds": gc.get_threshold(),
                }

            initial_gc_collected = 0
            if auto_gc:
                initial_gc_collected = gc.collect()

            memory_info_before = process.memory_info()
            memory_before_mb = memory_info_before.rss / (1024 * 1024)

            peak_memory_mb = memory_before_mb if track_peak else None
            memory_samples = []

            start_time = time.time()

            exception_raised = None
            result = None

            try:
                if auto_gc:
                    gc_was_enabled = gc.isenabled()
                    if gc_was_enabled:
                        gc.disable()

                try:
                    result = func(*args, **kwargs)

                    if track_peak:
                        current_mem = process.memory_info().rss / (1024 * 1024)
                        memory_samples.append(current_mem)
                        peak_memory_mb = max(peak_memory_mb, current_mem)

                finally:
                    if auto_gc and gc_was_enabled:
                        gc.enable()

            except Exception as e:
                exception_raised = e

            execution_time = time.time() - start_time

            memory_info_after_exec = process.memory_info()
            memory_after_exec_mb = memory_info_after_exec.rss / (1024 * 1024)

            if track_peak:
                peak_memory_mb = max(peak_memory_mb, memory_after_exec_mb)

            gc_objects_collected = 0
            if auto_gc:
                gc_objects_collected = gc.collect(2)
                gc_objects_collected += gc.collect()

            memory_info_final = process.memory_info()
            memory_after_mb = memory_info_final.rss / (1024 * 1024)

            memory_saved_mb = max(0.0, memory_after_exec_mb - memory_after_mb)

            threshold_exceeded = memory_after_mb > threshold_mb

            gc_stats = None
            if detailed_stats:
                gc_stats_after = {
                    "generations": gc.get_count(),
                    "stats": (
                        gc.get_stats() if hasattr(gc, "get_stats") else None
                    ),
                    "garbage": len(gc.garbage),
                    "thresholds": gc.get_threshold(),
                }
                gc_stats = {
                    "before": gc_stats_before,
                    "after": gc_stats_after,
                    "objects_collected": gc_objects_collected,
                    "initial_collection": initial_gc_collected,
                }

            report = ProfileReport(
                function_name=func.__name__,
                execution_time=execution_time,
                memory_before=memory_before_mb,
                memory_after=memory_after_mb,
                memory_saved=memory_saved_mb,
                gc_objects_collected=gc_objects_collected,
                exceeded_threshold=threshold_exceeded,
                peak_memory=peak_memory_mb if track_peak else None,
                gc_stats=gc_stats if detailed_stats else None,
            )

            if not hasattr(wrapper, "profile_reports"):
                wrapper.profile_reports = []
            wrapper.profile_reports.append(report)
            wrapper.last_report = report

            if exception_raised:
                raise exception_raised

            return result

        wrapper.profiler_config = {
            "auto_gc": auto_gc,
            "detailed_stats": detailed_stats,
            "track_peak": track_peak,
            "threshold_mb": threshold_mb,
        }

        wrapper.profile_reports = []
        wrapper.last_report = None

        return wrapper

    return decorator


def profile_functions(
    functions: List[Callable], config: Dict[str, Any]
) -> List[ProfileReport]:
    """
    Profile a list of functions with a given configuration.

    Args:
        functions: List of callable functions to profile
        config: Dictionary with profiling configuration

    Returns:
        List of ProfileReport objects for each function
    """
    reports = []

    auto_gc = config.get("auto_gc", False)
    detailed_stats = config.get("detailed_stats", False)
    track_peak = config.get("track_peak", False)
    threshold_mb = config.get("threshold_mb", float("inf"))

    for func in functions:
        profiled_func = memory_profiler(
            auto_gc=auto_gc,
            detailed_stats=detailed_stats,
            track_peak=track_peak,
            threshold_mb=threshold_mb,
        )(func)

        try:
            profiled_func()
        except Exception:
            pass

        if hasattr(profiled_func, "last_report") and profiled_func.last_report:
            reports.append(profiled_func.last_report)

    return reports


def example_process_data():
    """Allocate memory as an example."""
    data = [[i] * 1000 for i in range(1000)]
    return len(data)


def example_circular_refs():
    """Create circular references as an example."""
    objects = []
    for i in range(100):
        obj = {"id": i, "refs": objects}
        objects.append(obj)
    return objects


if __name__ == "__main__":
    @memory_profiler(auto_gc=True, threshold_mb=100)
    def process_data():
        """Process some data."""
        data = [[i] * 1000 for i in range(1000)]
        return len(data)

    result = process_data()
    print(f"# Output: {process_data.last_report}")

    @memory_profiler(detailed_stats=True, track_peak=True)
    def create_circular_refs():
        """Create circular references."""
        objects = []
        for i in range(100):
            obj = {"id": i, "refs": objects}
            objects.append(obj)
        return objects

    result = create_circular_refs()
    print(f"# Output: {create_circular_refs.last_report}")



In [ ]:
# tests

"""
Test suite for Python memory profiler system.

This module contains comprehensive unit tests for the memory profiler
decorator that uses the gc module to control garbage collection and
optimize memory usage during function execution.
"""

import gc
import unittest
from unittest.mock import Mock, patch
from dataclasses import dataclass
from main import memory_profiler


@dataclass
class ProfileReport:
    """Data class representing memory profiling report."""

    function_name: str
    execution_time: float
    memory_before: float
    memory_after: float
    memory_saved: float
    gc_objects_collected: int
    exceeded_threshold: bool


class TestMemoryProfiler(unittest.TestCase):
    """Test cases for memory profiler decorator functionality."""

    def setUp(self):
        """Set up test fixtures before each test method."""
        self.mock_psutil_process = Mock()
        self.mock_psutil_process.memory_info.return_value.rss = 50 * 1024 * 1024

    def tearDown(self):
        """Clean up after each test method."""
        gc.collect()

    @patch('psutil.Process')
    @patch('gc.collect')
    def test_basic_memory_profiling_without_auto_gc(self, mock_gc_collect,
                                                   mock_process):
        """Test basic memory profiling without automatic garbage collection."""
        mock_process.return_value = self.mock_psutil_process
        mock_gc_collect.return_value = 0

        @memory_profiler(auto_gc=False)
        def simple_function():
            """Simple test function."""
            return 42

        result = simple_function()

        # Verify function execution
        self.assertEqual(result, 42)

        # Verify gc.collect was not called when auto_gc=False
        mock_gc_collect.assert_not_called()

    @patch('psutil.Process')
    @patch('gc.collect')
    def test_memory_profiling_with_auto_gc_enabled(self, mock_gc_collect,
                                                  mock_process):
        """Test memory profiling with automatic garbage collection enabled."""
        mock_process.return_value = self.mock_psutil_process
        mock_gc_collect.return_value = 100

        @memory_profiler(auto_gc=True)
        def memory_intensive_function():
            """Function that creates memory-intensive objects."""
            data = [[i] * 100 for i in range(100)]
            return len(data)

        result = memory_intensive_function()

        # Verify function execution
        self.assertEqual(result, 100)

        # Verify gc.collect was called (at least once for cleanup)
        self.assertGreaterEqual(mock_gc_collect.call_count, 1)

    @patch('psutil.Process')
    @patch('gc.collect')
    def test_memory_threshold_exceeded_warning(self, mock_gc_collect,
                                              mock_process):
        """Test memory threshold exceeded warning functionality."""
        # Set up mock to simulate high memory usage
        mock_process.return_value.memory_info.return_value.rss = (
            150 * 1024 * 1024  # 150MB
        )
        mock_gc_collect.return_value = 50

        @memory_profiler(threshold_mb=100.0)
        def high_memory_function():
            """Function that exceeds memory threshold."""
            return "high memory usage"

        result = high_memory_function()

        # Verify function execution
        self.assertEqual(result, "high memory usage")

    @patch('psutil.Process')
    @patch('gc.collect')
    def test_memory_threshold_not_exceeded(self, mock_gc_collect,
                                          mock_process):
        """Test memory profiling when threshold is not exceeded."""
        mock_process.return_value = self.mock_psutil_process
        mock_gc_collect.return_value = 25

        @memory_profiler(threshold_mb=100.0)
        def low_memory_function():
            """Function that stays within memory threshold."""
            return "low memory usage"

        result = low_memory_function()

        # Verify function execution
        self.assertEqual(result, "low memory usage")

    @patch('psutil.Process')
    @patch('gc.collect')
    def test_detailed_stats_enabled(self, mock_gc_collect, mock_process):
        """Test memory profiling with detailed statistics enabled."""
        mock_process.return_value = self.mock_psutil_process
        mock_gc_collect.return_value = 75

        @memory_profiler(detailed_stats=True)
        def stats_function():
            """Function to test detailed statistics."""
            return {"status": "detailed"}

        result = stats_function()

        # Verify function execution
        self.assertEqual(result, {"status": "detailed"})

    @patch('psutil.Process')
    @patch('gc.collect')
    def test_peak_memory_tracking_enabled(self, mock_gc_collect,
                                         mock_process):
        """Test memory profiling with peak memory tracking enabled."""
        mock_process.return_value = self.mock_psutil_process
        mock_gc_collect.return_value = 30

        @memory_profiler(track_peak=True)
        def peak_tracking_function():
            """Function to test peak memory tracking."""
            temp_data = [i for i in range(1000)]
            return sum(temp_data)

        result = peak_tracking_function()

        # Verify function execution
        self.assertEqual(result, 499500)

    @patch('psutil.Process')
    @patch('gc.collect')
    def test_function_with_exception_handling(self, mock_gc_collect,
                                             mock_process):
        """Test memory profiling when function raises an exception."""
        mock_process.return_value = self.mock_psutil_process
        mock_gc_collect.return_value = 10

        @memory_profiler(auto_gc=True)
        def exception_function():
            """Function that raises an exception."""
            raise ValueError("Test exception")

        with self.assertRaises(ValueError):
            exception_function()

        # Verify gc.collect was still called for cleanup
        mock_gc_collect.assert_called()

    @patch('psutil.Process')
    @patch('gc.collect')
    def test_recursive_function_profiling(self, mock_gc_collect,
                                         mock_process):
        """Test memory profiling with recursive function calls."""
        mock_process.return_value = self.mock_psutil_process
        mock_gc_collect.return_value = 5

        @memory_profiler(auto_gc=True)
        def recursive_function(n):
            """Recursive function for testing."""
            if n <= 0:
                return 1
            return n * recursive_function(n - 1)

        result = recursive_function(5)

        # Verify function execution
        self.assertEqual(result, 120)

    @patch('psutil.Process')
    @patch('gc.collect')
    def test_zero_execution_time_handling(self, mock_gc_collect,
                                         mock_process):
        """Test handling of functions with extremely short execution times."""
        mock_process.return_value = self.mock_psutil_process
        mock_gc_collect.return_value = 0

        @memory_profiler(auto_gc=False)
        def instant_function():
            """Function with minimal execution time."""
            return None

        result = instant_function()

        # Verify function execution
        self.assertIsNone(result)

    @patch('psutil.Process')
    @patch('gc.collect')
    def test_no_memory_allocation_function(self, mock_gc_collect,
                                          mock_process):
        """Test profiling function that allocates no additional memory."""
        mock_process.return_value = self.mock_psutil_process
        mock_gc_collect.return_value = 0

        @memory_profiler(auto_gc=True)
        def no_allocation_function():
            """Function that doesn't allocate memory."""
            x = 1
            y = 2
            return x + y

        result = no_allocation_function()

        # Verify function execution
        self.assertEqual(result, 3)

    @patch('psutil.Process')
    @patch('gc.collect')
    def test_memory_decrease_during_execution(self, mock_gc_collect,
                                             mock_process):
        """Test handling when memory usage decreases during execution."""
        # Mock decreasing memory usage with enough return values
        mock_memory_info_before = Mock()
        mock_memory_info_before.rss = 100 * 1024 * 1024  # 100MB
        mock_memory_info_after = Mock()
        mock_memory_info_after.rss = 80 * 1024 * 1024    # 80MB

        mock_process_instance = Mock()
        mock_process_instance.memory_info.side_effect = [
            mock_memory_info_before,
            mock_memory_info_after,
            mock_memory_info_after,  # Additional call for safety
        ]
        mock_process.return_value = mock_process_instance
        mock_gc_collect.return_value = 20

        @memory_profiler(auto_gc=True)
        def memory_decreasing_function():
            """Function that decreases memory usage."""
            return "memory freed"

        result = memory_decreasing_function()

        # Verify function execution
        self.assertEqual(result, "memory freed")

    @patch('psutil.Process')
    @patch('gc.collect')
    def test_negative_memory_threshold_handling(self, mock_gc_collect,
                                               mock_process):
        """Test handling of negative memory threshold values."""
        mock_process.return_value = self.mock_psutil_process
        mock_gc_collect.return_value = 15

        @memory_profiler(threshold_mb=-10.0)
        def negative_threshold_function():
            """Function with negative threshold."""
            return "negative threshold"

        result = negative_threshold_function()

        # Verify function execution
        self.assertEqual(result, "negative threshold")

    @patch('psutil.Process')
    @patch('gc.collect')
    def test_zero_memory_threshold_handling(self, mock_gc_collect,
                                           mock_process):
        """Test handling of zero memory threshold value."""
        mock_process.return_value = self.mock_psutil_process
        mock_gc_collect.return_value = 8

        @memory_profiler(threshold_mb=0.0)
        def zero_threshold_function():
            """Function with zero threshold."""
            return "zero threshold"

        result = zero_threshold_function()

        # Verify function execution
        self.assertEqual(result, "zero threshold")

    @patch('psutil.Process')
    @patch('gc.collect')
    def test_circular_reference_handling(self, mock_gc_collect,
                                        mock_process):
        """Test memory profiling with circular references."""
        mock_process.return_value = self.mock_psutil_process
        mock_gc_collect.return_value = 200

        @memory_profiler(auto_gc=True, detailed_stats=True)
        def circular_refs_function():
            """Function that creates circular references."""
            objects = []
            for i in range(10):
                obj = {'id': i, 'refs': objects}
                objects.append(obj)
            return len(objects)

        result = circular_refs_function()

        # Verify function execution
        self.assertEqual(result, 10)

        # Verify gc.collect was called for circular reference cleanup
        mock_gc_collect.assert_called()

    @patch('psutil.Process')
    @patch('gc.collect')
    def test_combined_configuration_options(self, mock_gc_collect,
                                           mock_process):
        """Test memory profiling with multiple configuration options."""
        mock_process.return_value = self.mock_psutil_process
        mock_gc_collect.return_value = 150

        @memory_profiler(auto_gc=True, detailed_stats=True,
                        track_peak=True, threshold_mb=50.0)
        def combined_config_function():
            """Function with all configuration options enabled."""
            data = {'large_list': [i**2 for i in range(100)]}
            return len(data['large_list'])

        result = combined_config_function()

        # Verify function execution
        self.assertEqual(result, 100)

        # Verify gc.collect was called
        mock_gc_collect.assert_called()


# Model Breaking Proof

#### Model Breaking Task URL: <Add the URL here>

#### Model code:

```python
# code generated by the model

To develop a memory profiler system in Python that leverages the `gc` module for manual garbage collection and uses the `psutil` library for monitoring memory usage, we can create a decorator called `memory_profiler`. This decorator will be responsible for controlling garbage collection, tracking memory usage before and after the function execution, and reporting the memory profile statistics as a structured output.

Here's a step-by-step implementation of such a memory profiler:

### Step 1: Install Necessary Libraries
Ensure the required Python packages are installed:
```bash
pip install psutil
```

### Step 2: Define the ProfileReport
Define a data class `ProfileReport` to encapsulate memory profiling results:

```python
from dataclasses import dataclass

@dataclass
class ProfileReport:
    function_name: str
    execution_time: float
    memory_before: float
    memory_after: float
    memory_saved: float
    gc_objects_collected: int
    exceeded_threshold: bool
```

### Step 3: Implement the Memory Profiler Decorator
Implement the `memory_profiler` decorator to manage garbage collection and report memory usage:

```python
import time
import gc
import psutil
from functools import wraps

# <Issue>: Wrong input format - should be a function that accepts list of functions + config, not a decorator
def memory_profiler(auto_gc=False, detailed_stats=False, track_peak=False, threshold_mb=0):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            process = psutil.Process()
            
            # Memory before execution
            memory_before = process.memory_info().rss / (1024 * 1024)  # Convert to MB
            
            # Optionally run garbage collection before execution
            if auto_gc:
                gc.collect()
                memory_before_post_gc = process.memory_info().rss / (1024 * 1024)
                gc_saved_pre = memory_before - memory_before_post_gc
                memory_before = memory_before_post_gc
            
            # Start time measurement
            start_time = time.perf_counter()
            
            # Execute the function
            try:
                result = func(*args, **kwargs)
            except Exception as e:
                result = None
                print(f"Exception raised during function execution: {e}")
            
            # End time measurement
            end_time = time.perf_counter()
            execution_time = end_time - start_time
            
            # Optionally run garbage collection after execution
            gc_collected = 0
            memory_after = process.memory_info().rss / (1024 * 1024)
            if auto_gc:
                gc.collect()
                gc_collected = len(gc.get_objects())
                memory_after_gc = process.memory_info().rss / (1024 * 1024)
                gc_saved_post = memory_after - memory_after_gc
                memory_after = memory_after_gc
            
             <Issue>: Ignores gc_saved_pre and gc_saved_post calculations
            # Calculate memory statistics
            memory_saved = (memory_before - memory_after) if auto_gc else 0
            exceeded_threshold = (memory_after - memory_before) > threshold_mb

            # Create profile report
            report = ProfileReport(
                function_name=func.__name__,
                execution_time=execution_time,
                memory_before=memory_before,
                memory_after=memory_after,
                memory_saved=(gc_saved_pre if auto_gc else 0) + (gc_saved_post if auto_gc else 0),
                gc_objects_collected=gc_collected,
                exceeded_threshold=exceeded_threshold
            )
            
            # Optionally, print detailed stats
            if detailed_stats:
                gc_stats = gc.get_stats()
                print(f"GC Stats: {gc_stats}")
            
            return report
        
        return wrapper
    return decorator
```

### Step 4: Usage Example
Below are examples of using the `memory_profiler` decorator with configuration:

```python
@memory_profiler(auto_gc=True, threshold_mb=100)
def process_data():
    data = [[i]*1000 for i in range(1000)]
    return len(data)

@memory_profiler(detailed_stats=True, threshold_mb=1)
def create_circular_refs():
    objects = []
    for i in range(100):
        obj = {'id': i, 'refs': objects}
        objects.append(obj)
    return objects

# Example calls
report1 = process_data()
print(report1)

report2 = create_circular_refs()
print(report2)
```

### Important Considerations:

- **Accuracy**: The memory profiler provides approximate measurements due to how the Python memory manager and garbage collector work.
  
- **Overhead**: Tracking memory usage and executing garbage collection can introduce overhead, so careful consideration of these options is needed in performance-sensitive environments.

- **Peak Memory Tracking**: Implementing `track_peak` would require temporally storing memory samples throughout the execution or using an external tool.

This implementation provides a robust starting point for a production-ready memory profiler system in Python. Adjust error handling and logging details to fit specific production needs.
```